In [146]:
""" This notebook performs SAR instrument design based on a multi-objective optimization algorithm (NSGA-II), enabled by the 
    `pymoo 0.4.2` package. A baseline instrument is defined, and some of the instrument parameters are kept as optmization variables.
    Multiple objectives are defined in terms of the datametrics produced by the InstruPy package. 
    The result is a multi-dimensional Pareto-curve illustrating the trade-offs between different instrument designs. Any point on the
    Pareto curve is optimal and a suitable point can be selected by the user for implementation.

    Baseline instrument: Synthetic Aperture Radar with following fixed parameters:
                            * Operational altitude is 350km
                            * Dual-polarization (SMAP pulse configuration with default pulse-separation)
                            * L-band
                            * Operating points to be considered at incidence angles of 30 deg, 45 deg and 60 deg
                            * 1kW transmit power
                            * Full swath configuration (processed swath = illuminated swath)
                            * Losses, Noise: Antenna efficiency of 60%, 2dB radar loss, 2dB system noise figure

    Variables: (1) Daz [m] (antenna height in meters) range: 0.5m to 15m
               (2) Delv [m] (antenna width in meters) range: 0.5m to 5m
               (3) Chirp BW [MHz] (chirp bandwidth in Mega Hertz) range: 0.1MHz to 40MHz
               (4) Pulse width [us] (transmit pulse width in micro-seconds) range: 1us to 1000us
    
    (Minimization) Objectives: 
                (1) Antenna area: Antenna height * Antenna width
                
                (2) -1* Swath Width
                
                (3) Sigma NESZ [dB]
                
                (4) Speckle reduction [dB] for 1km2 pixel:
                    Number of Looks (N) = processed pixel area (1km2) / unprocessed pixel area (along-track resolution * cross-track resolution)
                    The speckle reduction in decibels is given by 10 log_10(1/N) (See 5.8, Pg 187, David Gardner Long , Fawwaz T. Ulaby - Microwave Radar and Radiometric Remote Sensing)
    
    Constraints: Valid operating point (not all designs are valid for SAR since a suitable operational PRF may not be available.)
                 The operations at incidence angles of 35 deg, 45 deg and 55 deg are considered.

"""
import numpy as np
import os, shutil
import matplotlib.pyplot as plt
import pickle

from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_problem, get_sampling, get_crossover, get_mutation, get_termination
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter
from pymoo.util.misc import stack
from pymoo.model.problem import Problem
from instrupy.synthetic_aperture_radar_model import SyntheticApertureRadar

Re = 6378.137e3 # [m]
c = 299792458 # m/s
h = 350e3 # [m]
inc = [30, 45, 60] # [deg]
orb_speed = np.sqrt(3.986004418e14/(Re + h)) # [m/s]

out_dir = os.path.dirname(os.path.realpath(__file__)) + '/moo_results/'
if os.path.exists(out_dir):
    shutil.rmtree(out_dir)
os.makedirs(out_dir)

SyntaxError: invalid syntax (3179498785.py, line 37)

In [147]:
class MyProblem(Problem):
    """     # Variables: (1) Daz [m], (2) Delv [m], (3), Chirp BW [MHz] (4) Pulse width [us]
            # Objectives: (1) Antenna area (2) -1* Swath Width (3) Sigma NESZ [dB] (4) # Speckle reduction [dB] for 1km2 pixel
            # Constrtains: Valid operating point
    """
    def __init__(self):
        super().__init__(n_var=4,
                         n_obj=4,
                         n_constr=1,
                         xl=np.array([0.5, 0.5, 0.1, 1]),
                         xu=np.array([15, 5, 40, 1000]),
                         elementwise_evaluation=True)

    @staticmethod
    def define_test_sar(daz_m, delv_m, chirpbw_Mhz, pulse_w_us):

        test_sar = SyntheticApertureRadar.from_json(  '{"@type": "Synthetic Aperture Radar",'
                                                '"orientation": {'
                                                '    "convention": "SIDE_LOOK",'
                                                '    "sideLookAngle": 55.21888' # not of significance in this problem
                                                '},'
                                                '"pulseWidth":'+ str(pulse_w_us*1e-6) +','
                                                '"antenna":{"shape": "RECTANGULAR", "height":'+ str(daz_m) +',"width":'+ str(delv_m) +',' 
                                                '            "apertureEfficiency": 0.6, "apertureExcitationProfile": "UNIFORM"},'
                                                '"operatingFrequency": 1.2757e9,' 
                                                '"peakTransmitPower": 1000,' 
                                                '"chirpBandwidth":'+ str(chirpbw_Mhz*1e6) +','     
                                                '"minimumPRF": 1,' 
                                                '"maximumPRF": 20000,' 
                                                '"radarLosses": 2,' 
                                                '"systemNoiseFigure": 2,'
                                                '"swathConfig": {'
                                                '   "@type": "full"'
                                                '},'
                                                '"polarization": {'
                                                '   "@type": "dual",'
                                                '   "pulseConfig": {'
                                                '        "@type": "SMAP"''}' 
                                                '}'                                                   
                                                '}')
        return test_sar            


    def _evaluate(self, x, out, *args, **kwargs):
        
        daz_m = x[0]
        delv_m = x[1]
        chirpbw_Mhz = x[2]
        pulse_w_us = x[3]

        test_sar = MyProblem.define_test_sar(daz_m, delv_m, chirpbw_Mhz, pulse_w_us)       
      
        # Calculate the metrics at the 3 incidence angles. A valid operation point must be found for all of them.
        obsv_metrics_1 = test_sar.calc_typ_data_metrics(alt_km = h*1e-3, v_sc_kmps = orb_speed*1e-3, v_g_kmps = orb_speed*1e-3*(Re/(Re+h)), 
                                                        incidence_angle_deg = inc[0], 
                                                        instru_look_angle_from_GP_inc_angle = True)
        
        obsv_metrics_2 = test_sar.calc_typ_data_metrics(alt_km = h*1e-3, v_sc_kmps = orb_speed*1e-3, v_g_kmps = orb_speed*1e-3*(Re/(Re+h)), 
                                                        incidence_angle_deg = inc[1], 
                                                        instru_look_angle_from_GP_inc_angle = True)

        obsv_metrics_3 = test_sar.calc_typ_data_metrics(alt_km = h*1e-3, v_sc_kmps = orb_speed*1e-3, v_g_kmps = orb_speed*1e-3*(Re/(Re+h)), 
                                                        incidence_angle_deg = inc[2], 
                                                        instru_look_angle_from_GP_inc_angle = True)


        # initialize to invalid observation point
        g1 = 1
        f1 = 1e9
        f2 = 1e9
        f3 = 1e9
        f4 = 1e9
            
        if(obsv_metrics_1["Sigma NESZ [dB]"] and obsv_metrics_2["Sigma NESZ [dB]"] and obsv_metrics_3["Sigma NESZ [dB]"] ):
            
            # Use below condition in case of fixed-swath simulations
            #if(obsv_metrics_1["(Nominal) Swath-width [km]"] == 25 and obsv_metrics_2["(Nominal) Swath-width [km]"] == 25 and obsv_metrics_3["(Nominal) Swath-width [km]"] == 25):
                
            # valid observation point
            g1 = -1

            f1 = daz_m * delv_m
            f2 = -1 * obsv_metrics_3["(Nominal) Swath-width [km]"]
            f3 = obsv_metrics_3["Sigma NESZ [dB]"]

            N_looks = 1e6 / (obsv_metrics_3["Ground Pixel Along-Track Resolution [m]"] * obsv_metrics_3["Ground Pixel Cross-Track Resolution [m]"])
            f4 = 10*np.log10(1/np.sqrt(N_looks))
            #print(f1,f2,f3,f4)
            



        out["F"] = np.column_stack([f1, f2, f3, f4])
        out["G"] = np.column_stack([g1])




NameError: name 'Problem' is not defined

In [148]:
problem = MyProblem()

algorithm = NSGA2(
    pop_size=100,
    sampling=get_sampling("real_random"),
    crossover=get_crossover("real_sbx", prob=0.9, eta=20),
    mutation=get_mutation("real_pm", eta=20),
    eliminate_duplicates=True
)

termination = get_termination("n_gen", 100)

res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               pf=problem.pareto_front(use_cache=False),
               save_history=True,
               verbose=False)

pickle.dump(res, open(out_dir+ "data.p", "wb" ) )
#res = pickle.load( open( "X.p", "rb" ) )

NameError: name 'MyProblem' is not defined

In [149]:
"""
# create the performance indicator object with reference point (4,4)
metric = Hypervolume(ref_point=np.array([15*15, 0, 0, 0]))

# collect the population in each generation
pop_each_gen = [a.pop for a in res.history]

# receive the population in each generation
obj_and_feasible_each_gen = [pop[pop.get("feasible")[:,0]].get("F") for pop in pop_each_gen]

# calculate for each generation the HV metric
hv = [metric.calc(f) for f in obj_and_feasible_each_gen]

# function evaluations at each snapshot
n_evals = np.array([a.evaluator.n_eval for a in res.history])
"""

'\n# create the performance indicator object with reference point (4,4)\nmetric = Hypervolume(ref_point=np.array([15*15, 0, 0, 0]))\n\n# collect the population in each generation\npop_each_gen = [a.pop for a in res.history]\n\n# receive the population in each generation\nobj_and_feasible_each_gen = [pop[pop.get("feasible")[:,0]].get("F") for pop in pop_each_gen]\n\n# calculate for each generation the HV metric\nhv = [metric.calc(f) for f in obj_and_feasible_each_gen]\n\n# function evaluations at each snapshot\nn_evals = np.array([a.evaluator.n_eval for a in res.history])\n'

In [150]:
# Design Space
plot = Scatter(title = "Design Space", axis_labels="x", tight_layout=True)
plot.add(res.X, s=30, facecolors='none', edgecolors='c')
plot.do()
plt.savefig(out_dir+"design_space.svg")
#plot.apply(lambda ax: ax.set_xlim(0.5, 0.5, 0.1, 1))
#plot.apply(lambda ax: ax.set_ylim(20, 20, 80, 1000))
plot.show()
# f1: Daz [m], f2: Delv [m], f3: Chirp BW [MHz], f4: Pulse width [us]



NameError: name 'Scatter' is not defined

In [151]:
# Objective Space
plot = Scatter(title = "Objective Space", tight_layout=True)
plot.add(res.F)
plot.do()
plt.savefig(out_dir+"obj_space.svg")
plot.show()

# f1: Antenna area [m2], f2: -Swath width [km], f3: Sigma NESZ [dB], f4: Speckle reduction [dB]

NameError: name 'Scatter' is not defined

In [152]:
import pandas as pd 
df = pd.DataFrame({"Daz [m]" : res.X[:,0], "Delv [m]" : res.X[:,1], "Chirp BW [MHz]" : res.X[:,2], "Pulse width [us]" : res.X[:,3], "Antenna area [m2]" : res.F[:,0], "Swath width [km]" : -1*res.F[:,1], "Sigma NESZ [dB]" : res.F[:,2], "Speckle reduction [dB]" : res.F[:,3]})
df.to_csv(out_dir+"pareto_front.csv")

NameError: name 'res' is not defined

In [153]:
"""
plt.plot(n_evals, hv, '-o')
plt.title("Convergence")
plt.xlabel("Function Evaluations")
plt.ylabel("Hypervolume")
plt.savefig("hypervolume.svg")
plt.show()
"""

'\nplt.plot(n_evals, hv, \'-o\')\nplt.title("Convergence")\nplt.xlabel("Function Evaluations")\nplt.ylabel("Hypervolume")\nplt.savefig("hypervolume.svg")\nplt.show()\n'

In [154]:
from pymoo.visualization.pcp import PCP
plot = PCP().add(res.F)
plot.add(res.F[0], linewidth=5, color="green")
plot.add(res.F[8], linewidth=5, color="cyan")
plot.add(res.F[163], linewidth=5, color="red")
plot.add(res.F[190], linewidth=5, color="black")
plot.add(res.F[519], linewidth=5, color="yellow")
plot.show()
plt.savefig(out_dir+"pcp.svg")


ModuleNotFoundError: No module named 'pymoo'

In [155]:
#f1: Antenna area [m2], f2: -Swath width [km], f3: Sigma NESZ [dB], f4: Speckle reduction [dB]
from pymoo.factory import get_performance_indicator
A = res.F[10]
hv = get_performance_indicator("hv", ref_point=np.array([15*15, 0, 0, 0]))
print("hv", hv.calc(A))

ModuleNotFoundError: No module named 'pymoo'

In [156]:
res.F[10]

NameError: name 'res' is not defined

In [157]:
res.X

NameError: name 'res' is not defined

In [158]:
res.F[1]

NameError: name 'res' is not defined

In [159]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

np.set_printoptions(threshold=np.inf)
res.F[res.F[:,2] < -30]

NameError: name 'res' is not defined

In [160]:
np.where(res.F[:,2] < -30)

NameError: name 'res' is not defined

In [161]:
res.F[84]

NameError: name 'res' is not defined

In [162]:
res.X[84]

NameError: name 'res' is not defined

In [163]:
np.size(res.F)

NameError: name 'res' is not defined